In [2]:
import json
import pandas as pd
import numpy as np
import sys

pd.set_option('display.max_colwidth', None)
sys.path.append('./src-py')

In [3]:
import sbert_training

In [4]:
from sentence_transformers import SentenceTransformer, InputExample, LoggingHandler, losses, models, util
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryClassificationEvaluator
from torch.utils.data import DataLoader
from sentence_transformers.evaluation import TripletEvaluator
from datetime import datetime
from zipfile import ZipFile

from sentence_transformers.datasets import SentenceLabelDataset
from sentence_transformers.datasets import NoDuplicatesDataLoader

import logging

logging.basicConfig(format='%(asctime)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO,
                    handlers=[LoggingHandler()])
logger = logging.getLogger(__name__)

In [5]:
taska_training_df = pd.read_csv('../data/TaskA_train.csv')
taskb_training_df = pd.read_csv('../data/TaskB_train.csv')

taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')
taskb_valid_df = pd.read_csv('../data/TaskB_dev.csv')

In [6]:
#Mapping labels
taska_training_df = taska_training_df[taska_training_df.Validity != 0]
taska_valid_df    = taska_valid_df[taska_valid_df.Validity != 0]

taska_training_df['label'] = taska_training_df.Validity.apply(lambda x : 1 if x == 1 else 0)
taska_valid_df['label'] = taska_valid_df.Validity.apply(lambda x : 1 if x == 1 else 0)

taska_training_df['Premise'] = taska_training_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)
taska_valid_df['Premise'] = taska_valid_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)

In [7]:
taska_training_df['input_txt'] = taska_training_df.apply(lambda x: '[CLS] {} [SEP] {} [SEP] {} [SEP]'.format(x['topic'], x['Premise'], x['Conclusion']), axis=1)

In [8]:
taska_training_df.head()

,topic,Premise,Conclusion,Validity,Validity-Confidence,Novelty,Novelty-Confidence,label,input_txt
0,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Depression is a well-known psychological problem of modern society that is partly caused by TV watching:,1,confident,1,confident,1,"[CLS] TV viewing is harmful to children [SEP] TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions. [SEP] Depression is a well-known psychological problem of modern society that is partly caused by TV watching: [SEP]"
1,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Children's TV viewing fosters negative emotions,1,very confident,-1,majority,1,"[CLS] TV viewing is harmful to children [SEP] TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions. [SEP] Children's TV viewing fosters negative emotions [SEP]"
2,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Popularity of TV is harmful to children,1,very confident,1,majority,1,"[CLS] TV viewing is harmful to children [SEP] TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions. [SEP] Popularity of TV is harmful to children [SEP]"
3,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",Violence on TV and in movies encourages psychological stress,1,very confident,1,majority,1,"[CLS] TV viewing is harmful to children [SEP] TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions. [SEP] Violence on TV and in movies encourages psychological stress [SEP]"
4,TV viewing is harmful to children,"TV viewing is harmful to children : The popularity of TV watching is among the reasons of this phenomenon. Violence, aggression, crimes and wars are broadcast through the daily news as well as in movies, showing dark pictures that encourage psychological tension, pessimism and negative emotions.",US-India deal does not cap or limit Indian fissile material production.,-1,very confident,-1,very confident,0,"[CLS] TV viewing is harmful to children [SEP] TV viewing is har

In [9]:
taska_training_df.label.value_counts()

1    401
0    320
Name: label, dtype: int64

In [10]:
taska_valid_df.label.value_counts()

1    125
0     74
Name: label, dtype: int64

In [27]:
def get_training_examples(df, eval_df, loss):
    
    logger.info("Read Triplet train dataset")
    train_examples = []
    for idx, row in df.iterrows():
        if loss == 'ContrastiveLoss':
            train_examples.append(InputExample(texts=[row['Premise'], row['Conclusion']], label=row['label']))
        elif loss == 'MultipleNegativesRankingLoss':
            if row['label'] == 1:
                train_examples.append(InputExample(texts=[row['Premise'], row['Conclusion']], label=1))
        else:
            train_examples.append(InputExample(texts=[row['anchor'], row['pos'], row['neg']], label=0))
            
    
    dev_samples = []
    for idx, row in eval_df.iterrows():
        dev_samples.append(InputExample(texts=[row['Premise'], row['Conclusion']], label=row['label']))
    
    return train_examples, dev_samples

            
def train_model(df, eval_df, output_path, model_name, num_epochs=3, train_batch_size=16, model_suffix='', \
                data_file_suffix='', max_seq_length=256, 
                special_tokens=[], loss='Triplet', sentence_transformer=False):
    
    output_path = output_path + model_name+ "-" + model_suffix + "-"+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    if sentence_transformer:
        word_embedding_model = SentenceTransformer(model_name)
        word_embedding_model.max_seq_length = max_seq_length
    else:
        word_embedding_model = models.Transformer(model_name)
        word_embedding_model.max_seq_length = max_seq_length
    
    
    if len(special_tokens) > 0:
        word_embedding_model.tokenizer.add_tokens(special_tokens, special_tokens=True)
        word_embedding_model.resize_token_embeddings(len(word_embedding_model.tokenizer))
        
    # Apply mean pooling to get one fixed sized sentence vector
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                                   pooling_mode_mean_tokens=True,
                                   pooling_mode_cls_token=False,
                                   pooling_mode_max_tokens=False)

    model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


    train_examples, dev_samples = get_training_examples(df, eval_df, loss)

    print('Len of training: {}'.format(len(train_examples)))
    print('Len of Dev: {}'.format(len(dev_samples)))
    
    if loss == 'MultipleNegativesRankingLoss':
        # Special data loader that avoid duplicates within a batch
        train_dataloader = NoDuplicatesDataLoader(train_examples, batch_size=train_batch_size)
        # Our training loss
        train_loss = losses.MultipleNegativesRankingLoss(model)
    elif loss == 'ContrastiveLoss':
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
        train_loss = losses.ContrastiveLoss(model)
    else:
        train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
        train_loss = losses.TripletLoss(model)
    

    evaluator = BinaryClassificationEvaluator.from_input_examples(dev_samples, batch_size=train_batch_size, name='sts-dev')

    warmup_steps = int(len(train_dataloader) * num_epochs * 0.1) #10% of train data


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              save_best_model=True,
              checkpoint_save_steps=1000,
              checkpoint_save_total_limit=3,
              evaluation_steps=20,
              warmup_steps=warmup_steps,
              output_path=output_path)

In [12]:
train_model(taska_training_df, taska_valid_df, '../data/output/', 'sentence-transformers/nli-roberta-large', num_epochs=20, train_batch_size=16, model_suffix='', data_file_suffix='', max_seq_length=256, special_tokens=[], loss='ContrastiveLoss', sentence_transformer=False)

Downloading:   0%|          | 0.00/673 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/780k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

...

2022-06-25 14:49:25 - Binary Accuracy Evaluation of the model on sts-dev dataset after epoch 17:
2022-06-25 14:49:25 - Accuracy with Cosine-Similarity:           67.84	(Threshold: 0.6526)
2022-06-25 14:49:25 - F1 with Cosine-Similarity:                 79.49	(Threshold: 0.5336)
2022-06-25 14:49:25 - Precision with Cosine-Similarity:          66.31
2022-06-25 14:49:25 - Recall with Cosine-Similarity:             99.20
2022-06-25 14:49:25 - Average Precision with Cosine-Similarity:  76.88

2022-06-25 14:49:25 - Accuracy with Manhatten-Distance:           67.84	(Threshold: 640.6236)
2022-06-25 14:49:25 - F1 with Manhatten-Distance:                 79.35	(Threshold: 670.8201)
2022-06-25 14:49:25 - Precision with Manhatten-Distance:          66.49
2022-06-25 14:49:25 - Recall with Manhatten-Distance:             98.40
2022-06-25 14:49:25 - Average Precision with Manhatten-Distance:  76.64

2022-06-25 14:49:25 - Accuracy with Euclidean-Distance:           67.84	(Threshold: 26.1675)
2022-06-2

Iteration:   0%|          | 0/46 [00:00<?, ?it/s]

2022-06-25 14:49:32 - Binary Accuracy Evaluation of the model on sts-dev dataset after epoch 18:
2022-06-25 14:49:32 - Accuracy with Cosine-Similarity:           67.84	(Threshold: 0.6517)
2022-06-25 14:49:32 - F1 with Cosine-Similarity:                 79.49	(Threshold: 0.5324)
2022-06-25 14:49:32 - Precision with Cosine-Similarity:          66.31
2022-06-25 14:49:32 - Recall with Cosine-Similarity:             99.20
2022-06-25 14:49:32 - Average Precision with Cosine-Similarity:  77.18

2022-06-25 14:49:32 - Accuracy with Manhatten-Distance:           67.84	(Threshold: 642.7328)
2022-06-25 14:49:32 - F1 with Manhatten-Distance:                 79.23	(Threshold: 709.7477)
2022-06-25 14:49:32 - Precision with Manhatten-Distance:          65.96
2022-06-25 14:49:32 - Recall with Manhatten-Distance:             99.20
2022-06-25 14:49:32 - Average Precision with Manhatten-Distance:  76.72

2022-06-25 14:49:32 - Accuracy with Euclidean-Distance:           67.84	(Threshold: 26.2034)
2022-06-2

Iteration:   0%|          | 0/46 [00:00<?, ?it/s]

2022-06-25 14:49:39 - Binary Accuracy Evaluation of the model on sts-dev dataset after epoch 19:
2022-06-25 14:49:39 - Accuracy with Cosine-Similarity:           67.84	(Threshold: 0.6506)
2022-06-25 14:49:39 - F1 with Cosine-Similarity:                 79.49	(Threshold: 0.5303)
2022-06-25 14:49:39 - Precision with Cosine-Similarity:          66.31
2022-06-25 14:49:39 - Recall with Cosine-Similarity:             99.20
2022-06-25 14:49:39 - Average Precision with Cosine-Similarity:  77.10

2022-06-25 14:49:39 - Accuracy with Manhatten-Distance:           68.34	(Threshold: 651.1418)
2022-06-25 14:49:39 - F1 with Manhatten-Distance:                 79.37	(Threshold: 745.2222)
2022-06-25 14:49:39 - Precision with Manhatten-Distance:          65.79
2022-06-25 14:49:39 - Recall with Manhatten-Distance:             100.00
2022-06-25 14:49:39 - Average Precision with Manhatten-Distance:  76.70

2022-06-25 14:49:39 - Accuracy with Euclidean-Distance:           67.84	(Threshold: 26.2521)
2022-06-

In [28]:
train_model(taska_training_df, taska_valid_df, '../data/output/', 
            'sentence-transformers/nli-roberta-large', 
            num_epochs=3, train_batch_size=8,
            model_suffix='ranking-loss', max_seq_length=256, special_tokens=[], 
            loss='MultipleNegativesRankingLoss', sentence_transformer=False)

2022-06-26 19:16:36 - Use pytorch device: cuda
2022-06-26 19:16:36 - Read Triplet train dataset
Len of training: 401
Len of Dev: 199


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

2022-06-26 19:16:39 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 0 after 20 steps:
2022-06-26 19:16:39 - Accuracy with Cosine-Similarity:           71.86	(Threshold: 0.7185)
2022-06-26 19:16:39 - F1 with Cosine-Similarity:                 79.45	(Threshold: 0.6203)
2022-06-26 19:16:39 - Precision with Cosine-Similarity:          69.46
2022-06-26 19:16:39 - Recall with Cosine-Similarity:             92.80
2022-06-26 19:16:39 - Average Precision with Cosine-Similarity:  83.44

2022-06-26 19:16:39 - Accuracy with Manhatten-Distance:           74.87	(Threshold: 598.0983)
2022-06-26 19:16:39 - F1 with Manhatten-Distance:                 80.92	(Threshold: 611.8794)
2022-06-26 19:16:39 - Precision with Manhatten-Distance:          77.37
2022-06-26 19:16:39 - Recall with Manhatten-Distance:             84.80
2022-06-26 19:16:39 - Average Precision with Manhatten-Distance:  83.71

2022-06-26 19:16:39 - Accuracy with Euclidean-Distance:           72.36	(Threshold: 22.8899

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

2022-06-26 19:17:10 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 1 after 20 steps:
2022-06-26 19:17:10 - Accuracy with Cosine-Similarity:           66.83	(Threshold: 0.4866)
2022-06-26 19:17:10 - F1 with Cosine-Similarity:                 79.11	(Threshold: 0.4671)
2022-06-26 19:17:10 - Precision with Cosine-Similarity:          65.45
2022-06-26 19:17:10 - Recall with Cosine-Similarity:             100.00
2022-06-26 19:17:10 - Average Precision with Cosine-Similarity:  80.12

2022-06-26 19:17:10 - Accuracy with Manhatten-Distance:           67.34	(Threshold: 700.8440)
2022-06-26 19:17:10 - F1 with Manhatten-Distance:                 79.11	(Threshold: 803.3531)
2022-06-26 19:17:10 - Precision with Manhatten-Distance:          65.45
2022-06-26 19:17:10 - Recall with Manhatten-Distance:             100.00
2022-06-26 19:17:10 - Average Precision with Manhatten-Distance:  81.33

2022-06-26 19:17:10 - Accuracy with Euclidean-Distance:           66.83	(Threshold: 30.51

Iteration:   0%|          | 0/50 [00:00<?, ?it/s]

2022-06-26 19:17:18 - Binary Accuracy Evaluation of the model on sts-dev dataset in epoch 2 after 20 steps:
2022-06-26 19:17:18 - Accuracy with Cosine-Similarity:           66.33	(Threshold: 0.6001)
2022-06-26 19:17:18 - F1 with Cosine-Similarity:                 78.86	(Threshold: 0.4173)
2022-06-26 19:17:18 - Precision with Cosine-Similarity:          65.10
2022-06-26 19:17:18 - Recall with Cosine-Similarity:             100.00
2022-06-26 19:17:18 - Average Precision with Cosine-Similarity:  80.45

2022-06-26 19:17:18 - Accuracy with Manhatten-Distance:           67.84	(Threshold: 586.1477)
2022-06-26 19:17:18 - F1 with Manhatten-Distance:                 79.11	(Threshold: 820.2714)
2022-06-26 19:17:18 - Precision with Manhatten-Distance:          65.45
2022-06-26 19:17:18 - Recall with Manhatten-Distance:             100.00
2022-06-26 19:17:18 - Average Precision with Manhatten-Distance:  81.52

2022-06-26 19:17:18 - Accuracy with Euclidean-Distance:           66.83	(Threshold: 27.34

### Using the auto-generated conclusions:

In [ ]:
taska_training_df = pd.read_csv('../data/')
taska_valid_df = pd.read_csv('../data/TaskA_dev.csv')

In [ ]:
#Mapping labels
taska_training_df = taska_training_df[taska_training_df.Validity != 0]
taska_valid_df    = taska_valid_df[taska_valid_df.Validity != 0]

taska_training_df['label'] = taska_training_df.Validity.apply(lambda x : 1 if x == 1 else 0)
taska_valid_df['label'] = taska_valid_df.Validity.apply(lambda x : 1 if x == 1 else 0)

taska_training_df['Premise'] = taska_training_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)
taska_valid_df['Premise'] = taska_valid_df.apply(lambda x: x['topic'] + ' : ' +  x['Premise'], axis=1)